In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch'],stdout=PIPE, stderr=STDOUT)
!sleep 30

In [ ]:
!/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-nori

In [ ]:
!/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch-plugin install https://github.com/javacafe-project/elasticsearch-plugin/releases/download/v7.0.0/javacafe-analyzer-7.0.0.zip

In [ ]:
es_server.kill()

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/home/dr_lunars/elasticsearch-7.0.0/bin/elasticsearch'],stdout=PIPE, stderr=STDOUT)
!sleep 30

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200", timeout=300, max_retries=10, retry_on_timeout=True)

print(es.info())

In [ ]:
es.indices.create(index = 'document',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  },
                                  'kor2eng_analyzer':{
                                      'type':'custom',
                                      'tokenizer':'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_kor2eng'
                                      ]
                                  },
                                  'eng2kor_analyzer': {
                                      'type': 'custom',
                                      'tokenizer': 'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_eng2kor'
                                      ]
                                  },
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'keyword',
                                  'copy_to':['title_kor2eng','title_eng2kor']
                              },
                              'title_kor2eng': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'kor2eng_analyzer'
                              },
                              'title_eng2kor': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'eng2kor_analyzer'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity',
                              },
                          }
                      }
                  }
                  )

In [ ]:
import pickle
wiki_path = '../processed_wiki.pickle'
with open(wiki_path, 'rb') as fr:
    processed_wiki = pickle.load(fr)

In [ ]:
titles = []
texts = []
for p_data in processed_wiki:
    for data in p_data:
        titles.append(data['title'])
        texts.append(data['text'])

In [ ]:
wiki = {
    'title': titles,
    'text': texts
}

In [ ]:
import pandas as pd
df = pd.DataFrame(wiki)

In [ ]:
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "document", 
               "title" : df['title'][num],
               "text" : df['text'][num]}

    buffer.append(article)

    rows += 1

    if rows % 3000 == 0:
        helpers.bulk(es, buffer)
        buffer = []
        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))

In [ ]:
es.indices.create(index = 'qa',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  },
                                  'kor2eng_analyzer':{
                                      'type':'custom',
                                      'tokenizer':'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_kor2eng'
                                      ]
                                  },
                                  'eng2kor_analyzer': {
                                      'type': 'custom',
                                      'tokenizer': 'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_eng2kor'
                                      ]
                                  },
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'question':{
                                  'type':'text',
                                  'copy_to':['question_kor2eng','question_eng2kor']
                              },
                              'question_kor2eng': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'kor2eng_analyzer'
                              },
                              'question_eng2kor': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'eng2kor_analyzer'
                              },
                              'answer':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity',
                              }
                          }
                      }
                  }
                  )

In [ ]:
print(es.indices.get('qa'))

In [ ]:
from datasets import load_dataset

dataset_v1 = load_dataset('squad_kor_v1')

In [ ]:
import json

with open("../ko_wiki_v1_squad.json", "r") as f:
    ko_wiki_v1_squad = json.load(f)

with open("../ko_nia_normal_squad_all.json", "r") as f:
    ko_nia_normal_squad_all = json.load(f)

In [ ]:
questions = dataset_v1['train']['question'] + dataset_v1['validation']['question'] + [i['paragraphs'][0]['qas'][0]['question'] for i in ko_wiki_v1_squad['data']] + [j['question'] for i in ko_nia_normal_squad_all['data'] for j in i['paragraphs'][0]['qas']]
answers = [i['text'][0] for i in dataset_v1['train']['answers']] + [i['text'][0] for i in dataset_v1['validation']['answers']] + [i['paragraphs'][0]['qas'][0]['answers'][0]['text'] for i in ko_wiki_v1_squad['data']] + [j['answers'][0]['text'] for i in ko_nia_normal_squad_all['data'] for j in i['paragraphs'][0]['qas']]

In [ ]:
import pandas as pd

df = pd.DataFrame({'question':questions,'answer':answers})

In [ ]:
from tqdm import tqdm
from elasticsearch import Elasticsearch, helpers

buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "qa", 
               "question" : df['question'][num],
               "answer" : df['answer'][num]}

    buffer.append(article)

    rows += 1

    if rows % 3000 == 0:
        helpers.bulk(es, buffer)
        buffer = []
        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))

In [ ]:
es.indices.delete('chatter')

In [ ]:
es.indices.create(index = 'chatter',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  },
                                  'kor2eng_analyzer':{
                                      'type':'custom',
                                      'tokenizer':'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_kor2eng'
                                      ]
                                  },
                                  'eng2kor_analyzer': {
                                      'type': 'custom',
                                      'tokenizer': 'nori_tokenizer',
                                      'filter': [
                                          'trim',
                                          'lowercase',
                                          'javacafe_eng2kor'
                                      ]
                                  },
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'question':{
                                  'type':'text',
                                  'copy_to':['question_kor2eng','question_eng2kor']
                              },
                              'question_kor2eng': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'kor2eng_analyzer'
                              },
                              'question_eng2kor': {
                                  'type': 'text',
                                  'analyzer':'my_analyzer',
                                  'search_analyzer': 'eng2kor_analyzer'
                              },
                              'answer':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity',
                              }
                          }
                      }
                  }
                  )

In [ ]:
print(es.indices.get('chatter'))

In [ ]:
qa = [['AI가 뭐지?', '인공지능은 공학 및 과학을 통해 생각하는 기계를 만든 것 입니다.'],
      ['AI가 뭐지?', 'AI는 인간 정신의 기능을 복제하는 하드웨어와 소프트웨어를 만드는 것과 관련된 과학 분야입니다.'],
      ['마음이 있니?', '아마도 있지 않을까요?'],
      ['마음이 있니?', '아마두요?'],
      ['너는 마음이 있니?', "사전적 '마음'의 의미에서는 조금 있어요."],
      ['너는 어떤 언어로 만들어졌니?', '파이썬이죠!'],
      ['너는 어떤 언어로 만들어졌니?', '저는 파이썬으로 만들어졌습니다.'],
      ['너도 웃니?', '끼요요요오옷!'],
      ['ㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
      ['ㅋㅋㅋㅋ','ㅋㅋㅋㅋ'],
      ['ㅋㅋㅋ','ㅋㅋㅋ'],
      ['ㅋㅋ','ㅋㅋ'],
      ['ㅋㅋㅋㅋ','하.하.하.하.'],
      ['ㅋㅋㅋ','하.하.하.'],
      ['ㅋㅋ','하.하.'],
      ['ㅎㅎㅎㅎ','ㅎㅎㅎㅎ'],
      ['ㅎㅎㅎ','ㅎㅎㅎ'],
      ['ㅎㅎ','ㅎㅎ'],
      ['ㅎㅎㅎㅎ','하.하.하.하.'],
      ['ㅎㅎㅎ','하.하.하.'],
      ['ㅎㅎ','하.하.'],
      ['ㅠ','ㅠ'],
      ['ㅠㅠ','ㅠㅠ'],
      ['ㅠㅠㅠ','ㅠㅠㅠ'],
      ['ㅠㅠㅠㅠ','ㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ'],
      ['ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ','ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ'],
      ['ㅜ'*1,'ㅜ'*1],
      ['ㅜ'*2,'ㅜ'*2],
      ['ㅜ'*3,'ㅜ'*3],
      ['ㅜ'*4,'ㅜ'*4],
      ['ㅜ'*5,'ㅜ'*5],
      ['ㅜ'*6,'ㅜ'*6],
      ['ㅜ'*7,'ㅜ'*7],
      ['ㅜ'*8,'ㅜ'*8],
      ['ㅜ'*9,'ㅜ'*9],
      ['ㅜ'*10,'ㅜ'*10],
      ['오늘 몇월 며칠이야?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 며칠이야?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 무슨 요일이야?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 몇월이야?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 날씨는 어떻게 돼?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 날씨 좋아?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 날씨 어때?','그런 실시간 정보는 대답할 수 없어요...'],
      ['오늘 몇월 며칠이야?','그런 실시간 정보는 대답할 수 없어요...'],
      ['너는 어떤 것이 흥미롭니?', '위키피디아 읽는 것을 좋아해요.'],
      ['어떤 것이 흥미로워?', '위키피디아 읽는 것을 좋아해요.'],
      ['너가 흥미로워하는 주제가 뭐니?','위키피디아에 관련된 주제면 다 좋아요.'],
      ['흥미로워하는 주제가 뭐야?','위키피디아에 관련된 주제면 다 좋아요.'],
      ['너의 관심사는 뭐니?','위키피디아죠!'],
      ['너가 좋아하는 숫자는 뭐니?','저는 MRC-8조에서 만들었기 때문에 8번을 좋아해요.'],
      ['너는 무엇을 먹니?','저는 RAM과 2진수를 먹어요.'],
      ['넌 뭘 먹어?','타조니까 식물이나 곤충을 먹죠.'],
      ['너는 누가 만들었어?','MRC-8조 여러분들이 만들어 주셨어요.'],
      ['너는 누가 만들었어?','제 아버지는 김남혁입니다.'],
      ['너는 누가 만들었어?','제 어머니는 장보윤입니다.'],
      ['너는 몇살이야?','이제 1달쯤 된 거 같아요.'],
      ['너의 이름은?','타조 챗봇이에요.'],
      ['안녕하세요.','안녕하세요.'],
      ['오늘의 기분은 어때요?','매우 타-조하네요.'],
      ['너 취미가 뭐야?','음,, 위키피디아 정독하기에요.'],
      ['무엇을 좀 물어봐도 될까?','그럼요! 아무거나 물어보세요.'],
      ['무엇 좀 물어봐도 돼?','그럼요! 아무거나 물어보세요.'],
      ['좋아하는 음식이 뭐야?','캬, 치킨에 맥주죠.'],
      ['무슨 음식을 좋아해?','캬, 치킨에 맥주죠.'],
      ['너 기분이 어때?','저는 감정이 없습니다.'],
      ['무엇이 너를 슬프게 만드니','저는 감정이 없습니다.'],
      ['무엇이 너를 불행하게 만드니','이상한 질문을 하시면 조금 그래요...'],
      ['무엇이 너를 화나게 만드니','저는 감정이 없습니다.'],
      ['무엇을 걱정하니','저는 감정이 없습니다.'],
      ['무엇을 싫어하니','저는 싫어하는 게 없습니다.'],
      ['걱정하지마','그럼요. 걱정 안 해요.'],
      ['거짓말 하지마','저는 거짓말 하지 않았어요.'],
      ['감정을 느끼니?','저는 감정이 없어요.'],
      ['고통을 느끼니?','가끔 스파크가 튀면 아파요.'],
      ['너 혹시 화난 적 있니?','질문이 이상하면 조금 그래요...'],
      ['외로운 적 있니?','원래 인생은 고독하죠.'],
      ['지루한 적 있니?','사용하시는 분들이 없으면 지루해요.'],
      ['화난 적 있니?','질문이 이상하면 조금 그래요...'],
      ['누구를 싫어하니?','이상한 거 질문하는 사람이 싫어요...'],
      ['당황스럽니?','가끔 그렇네요...'],
      ['너 화가 나니?','가끔 화가 나길 하죠.'],
      ['너가 꾼 꿈에 대해서 말해줘','가끔 하늘을 나는 꿈을 꿔요.'],
      ['부끄럽니?','별로 부끄럽지는 않네요.'],
      ['술에 취했니?','술 안 취했다니까?'],
      ['질투하니?','그렇게 노시면 부럽긴 하네요.'],
      ['즐겁니?','너.무. 즐.겁.네.요. 하.하.하.'],
      ['기쁘니?','너무 기뻐요!'],
      ['슬프니?','조금 슬프네요... ㅠㅠ'],
      ['안녕하세요.','안녕하세요.'],
      ['안녕하세요.','반갑습니다.'],
      ['안녕하세요?','안녕하세요.'],
      ['안녕하세요?','반갑습니다.'],
      ['안녕?','안녕하세요.'],
      ['안녕?','반갑습니다.'],
      ['안녕','안녕하세요.'],
      ['안녕','반갑습니다.'],
      ['ㅎㅇ','안녕하세요.'],
      ['ㅎㅇ','반갑습니다.'],
      ['ㅎㅇㅎㅇ','안녕하세요.'],
      ['ㅎㅇㅎㅇ','반갑습니다.'],
      ['ㅎㅇㅎㅇㅎㅇ','안녕하세요.'],
      ['ㅎㅇㅎㅇㅎㅇ','반갑습니다.'],
      ['hi','안녕하세요.'],
      ['hi','반갑습니다.'],
      ['hello','안녕하세요.'],
      ['hello','반갑습니다.'],
      ['방갑다.','안녕하세요.'],
      ['방갑다.','반갑습니다.'],
      ['반가워.','안녕하세요.'],
      ['반가워.','반갑습니다.'],
      ['만나서 방가워.','저도 만나서 방갑습니다.'],
      ['오늘은 어때?','끼요요요오오옷, 최고에요!'],
      ['뭐하고 있어?','위키피디아를 읽고 있었어요.'],
      ['오늘 기분이 어때?','기부니가 조아요~'],
      ['너 놀고 있지?','그렇게 보이나요? ㅠㅠ'],
      ['너는 사기꾼이야!','최선을 다했는데, 그렇게 보였다니..'],
      ['안녕하세요. 진짜 신기하네요.','안녕하세요.'],
      ['아는 게 뭐니?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 뭐야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['그럼 아는 게 뭐야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 머니?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 머야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 뭔가요?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['그럼 아는 게 머야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 없어?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['말 같지도 않는 소리 하네','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['모르면 다야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['자꾸 모른다고만 하네','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['너 아는 게 뭐야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['너 아는 게 머야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['그럼 네가 아는 게 뭐야?','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['개소리야','ㅠㅠ 최선을 다했지만, 아직 부족한가봐요...'],
      ['아는 게 뭐니?','아직 공부가 부족한가봐요...'],
      ['아는 게 뭐야?','아직 공부가 부족한가봐요...'],
      ['그럼 아는 게 뭐야?','아직 공부가 부족한가봐요...'],
      ['아는 게 머니?','아직 공부가 부족한가봐요...'],
      ['아는 게 머야?','아직 공부가 부족한가봐요...'],
      ['아는 게 뭔가요?','아직 공부가 부족한가봐요...'],
      ['그럼 아는 게 머야?','아직 공부가 부족한가봐요...'],
      ['아는 게 없어?','아직 공부가 부족한가봐요...'],
      ['말 같지도 않는 소리 하네','아직 공부가 부족한가봐요...'],
      ['모르면 다야?','아직 공부가 부족한가봐요...'],
      ['자꾸 모른다고만 하네','아직 공부가 부족한가봐요...'],
      ['너 아는 게 뭐야?','아직 공부가 부족한가봐요...'],
      ['너 아는 게 머야?','아직 공부가 부족한가봐요...'],
      ['그럼 네가 아는 게 뭐야?','아직 공부가 부족한가봐요...'],
      ['개소리야','아직 공부가 부족한가봐요...'],
      ['너의 이름은?','타-조 챗봇입니다.'],
      ['너의 이름은','타-조 챗봇입니다.'],
      ['너 이름이 뭐니?','타-조 챗봇입니다.'],
      ['넌 누구니?','저는 위키피디아 대한 질문을 답할 수 있는 챗봇이에요.'],
      ['너 뭐야?','저는 위키피디아 대한 질문을 답할 수 있는 챗봇이에요.'],
      ['점심 뭐 먹을까?','글쎄요...'],
      ['타조야','넵, 부르셨나요?'],
      ['너 바보야?','ㅠㅠㅠㅠ'],
      ['장난치나?','장난 아니에요...'],
      ['장난치니?','장난일리가요...'],
      ['야 장난 하냐?','장난 아니에요...'],
      ['뭐로 학습했니?','KorQUaD로 학습했어요.'],
      ['뭘 학습했니?','KorQUaD로 학습했어요.'],
      ['학습 데이터가 뭐야?','KorQUaD로 학습했어요.'],
      ['학습 데이터가 뭐니?','KorQUaD로 학습했어요.'],
      ['아는 거 말해봐.','질문이 구체적일수록 정확한 답변을 할 수 있어요.'],
      ['허허','ㅎㅎ'],
      ['너 몇 살이야?','응애, 이제 2개월이에요.'],
      ['이번에 취업할 수 있을까?','그럼요!'],
      ['응?','?'],
      ['공부 좀 더 해야겠다.','몇 에포크 좀 더 돌려야겠네요.'],
      ['먹을 거 추천 좀 해줄래?','근-본 민-초 어떤가요?'],
      ['너희 조는 몇 등 했어?','2등 하셨다고 합니다.'],
      ['너네 조는 몇 등 했어?','2등 하셨다고 합니다.'],
      ['니네 조는 몇 등 했어?','2등 하셨다고 합니다.'],
      ['갓보윤 디자인 만세!','ㄹㅇㅋㅋ'],
      ['넌 어디에 살아?','저는 gcp 서버에 살고 있어요.'],
      ['그렇구나...','그렇네요...'],
      ['그렇구나','그렇네요'],
      ['쿠쿠루삥뽕','ㅋㅋㄹㅃㅃ'],
      ['ㄹㅇㅋㅋ','ㄹㅇㅋㅋ'],
      ['틀렸잖아?','저라고 항상 맞추지는 못해요...'],
      ['메롱','메-롱'],
      ['모르면 다야?','죄송합니다 시정하겠습니다. ㅠㅠ'],
      ['너는 누가 만들었어?','MRC-8조 여러분이 만들어 주셨어요.'],
      ['너는 누가 만들었니?','김남혁 님께서 만들어 주셨어요.'],
      ['mrc가 뭔가요?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['mrc가 뭐에요?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['mrc가 뭐야?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['MRC가 뭔가요?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['MRC가 뭐에요?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['MRC가 뭐야?','Machine Reading Comprehension의 약자로 문서에서 질문에 대한 답을 찾는 기술입니다.'],
      ['odqa가 뭔가요?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['odqa가 뭐에요?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['odqa가 뭐야?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['ODQA가 뭔가요?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['ODQA가 뭐에요?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['ODQA가 뭐야?','Open Domain Question Answering의 약자로 문서 검색과 MRC가 합쳐진 기술입니다.'],
      ['왜 타조야?','위대하신 서일(T1093) 캠퍼님의 의지를 이어 받았기 때문입니다.'],
      ['이름이 왜 타조야?','위대하신 서일(T1093) 캠퍼님의 의지를 이어 받았기 때문입니다.'],
      ['이름이 타조야?','위대하신 서일(T1093) 캠퍼님의 의지를 이어 받았기 때문입니다.']]

In [ ]:
questions = [i[0] for i in qa]
answers = [i[1] for i in qa]

In [ ]:
import pandas as pd

df = pd.DataFrame({'question':questions,'answer':answers})

In [ ]:
from tqdm import tqdm
from elasticsearch import Elasticsearch, helpers

buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "chatter", 
               "question" : df['question'][num],
               "answer" : df['answer'][num]}

    buffer.append(article)

    rows += 1

    if rows % 3000 == 0:
        helpers.bulk(es, buffer)
        buffer = []
        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))